### **검색 알고리즘 구현 코드**

##### **활용 데이터**
* 카카오맵 맛집 크롤링 데이터 : kakao_food_deduplicated.csv 


* 촬영지 데이터(미디어타입 별 분할) :
    * drama : media_drama.csv
    * movie : media_movie.csv
    * artist : media_artist.csv

* 숙박 데이터 : 
    * hotel_pre.csv

##### **알고리즘 구조 설계**
* 카카오맵 맛집 데이터와 촬영지 데이터 기준 각 drama, movie, artist가 다른 DB에 저장되어있다고 가정 

1) drama/movie/artist 중 하나의 분야에 가서 배우명/아티스트명 검색 
2) 지역 14개중에 선택 -> area = ['충청', '대구', '대전', '강원', '경기', '인천', '제주', '전라', '세종', '서울', '울산', '부산', '경상', '광주'] -> 각 데이터의 '지역명' 컬럼에 저장되어 있음 
2) 해당 배우나 아티스트가 촬영한 데이터를 선택한 지역에 해당하는 media_drama, media_movie, media_artist에서 추출 -> 장소타입별 3개씩만 추출!
3) 추출된 촬영지 데이터 근처 여행코스와 숙박 추천 -> 최종 추천 목록 : 맛집 3개, 카페 3개, 관광지(playground) 3개, 숙박 3개 
3) - (1) 촬영지 데이터에서 추출된 데이터의 카테고리가 "playground"로만 이루어져있다면, 카카오맵 맛집 데이터에서 맛집 3개, 카페 3개, 숙박 데이터에서 숙박 3개 추천
3) - (2) 예를 들어 촬영지 데이터에서 추출된 데이터의 카테고리가 "restaurant" 2개, "cafe" 1개, "playground" 3개로 이루어져있다면, 카카오맵 맛집 데이터에서 맛집 1개, 카페 2개, 숙박 데이터 3개 추천해서 항상 최종 추천 수를 12개로 맞추는 걸로 !
4) 추천된 코스 지도 시각화 

##### 데이터 불러오기 

In [11]:
# 카카오맵 맛집 크롤링 데이터 
import pandas as pd

kakao_food = pd.read_csv("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/카카오맵 맛집 크롤링_전처리_데이터/kakao_food_geocode.csv", encoding="utf-8-sig")
kakao_food

,식당이름,카테고리,별점,리뷰 수,웹사이트,주소,지역명,위도,경도
0,덕수식당,restaurant,4.7,670,https://place.map.kakao.com/8891473,충남 태안군 태안읍 중앙로 133-8 1층(지번) 태안읍 동문리 339-15,충청,36.751762,126.303150
1,한일식당,restaurant,3.6,347,https://place.map.kakao.com/9573537,충남 예산군 삽교읍 삽교역로 58(지번) 삽교읍 신가리 20-14,충청,36.684627,126.749091
2,장춘닭개장,restaurant,4.2,187,https://place.map.kakao.com/11169052,충남 당진시 정안로 50 1층(지번) 원당동 381,충청,36.906543,126.649845
3,엄가네본가시골집,restaurant,3.7,284,https://place.map.kakao.com/13311506,충남 천안시 서북구 봉정로 168-2(지번) 성정동 142-6,충청,36.816957,127.142706
4,에이스식당,restaurant,4.8,109,https://place.map.kakao.com/9250070,충남 당진시 면천면 면천서문1길 70(지번) 면천면 성상리 768-1,충청,36.816980,126.666814
...,...,...,...,...,...,...,...,...,...
5436,모드니,cafe,4.6,54,https://place.map.kakao.com/6376422,광주 동구 동명로14번길 19 1층(지번) 동명동 73-19,광주,35.150059,126.925408
5437,엄니도시락한식뷔페 평동점,restaurant,4.0,15,https://place.map.kakao.com/206831746,광주 광산구 평동산단로 192 1층(지번) 월전동 1208,광주,35.123978,126.768528
5438,한옥식당,restaurant,3.9,50,https://place.map.kakao.com/21301951,광주 남구 백서로 87-2(지번) 양림동 137,광주,35.140323,126.915118
5439,프랭크버거 광주상무점,restaurant,3.9,29,https://place.map.kakao.com/1113376476,광주 서구 상무중앙로 29 1층(지번) 치평동 1178-6,광주,35.149591,126.847908


In [12]:
# 촬영지 데이터 : 드라마
media_drama = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_drama_pre.csv", encoding='utf-8-sig')
media_drama

,미디어타입,제목,장소명,장소타입,장소설명,주소,위도,경도,주소_지역명,배우이름
0,drama,(아는 건 별로 없지만) 가족입니다,커피파머,cafe,6회에서 김은희(한예리)와 안효석(이종원)이 윤태형(김태호)의 은신처에서 서울로 돌...,경기도 고양시 일산서구 대화로 61,37.667523,126.72817,경기,"['한예리', '이종원', '김태호']"
1,drama,(아는 건 별로 없지만) 가족입니다,셀렉토커피 남양주호평점,cafe,10회에서 하라(배윤경)는 은희(한예리)에게 이 커피숍에서 만나자고 메시지를 보낸다...,경기도 남양주시 천마산로 21,37.663558,127.24748,경기,"['배윤경', '한예리', '신동욱']"
2,drama,(아는 건 별로 없지만) 가족입니다,카페 그루비,cafe,"안효석(이종원)은 바리스타로, 김은주(추자현)는 단골손님이다. 1화에서 이진숙(원미...",경기도 수원시 영통구 센트럴파크로127번길 148,37.293983,127.05538,경기,"['이종원', '추자현', '원미경']"
3,drama,(아는 건 별로 없지만) 가족입니다,미리내 성지,playground,시누이 도혜지(황우슬혜)는 남편이 가문의 회사를 상속받을 수 있도록 그녀가 길을 잃...,경기도 안성시 양성면 미리내성지로 420,37.146102,127.25865,경기,"['황우슬혜', '고규필']"
4,drama,(아는 건 별로 없지만) 가족입니다,초은당,playground,김은희(한예리)와 동료 서경옥(가득희)은 1회에서 작가 의뢰인을 만나기 위해 이 한...,경기도 양평군 서종면 북한강로814번길 46-6,37.611484,127.35397,경기,"['한예리', '가득희', '신재하']"
...,...,...,...,...,...,...,...,...,...,...
7334,drama,You Raise Me Up (유 레이즈 미 업),서울마리나 클럽&요트,playground,6회에서 산책 후 밤에 이 클럽 마당에 한강 기슭에 앉아 그들은 키스를 나눈다.,서울특별시 영등포구 여의서로 160,37.534829,126.91157,서울,[]
7335,drama,You Raise Me Up (유 레이즈 미 업),한강대교,playground,2회에서 절망적이고 우울한 도용식(윤시윤)이 이 다리에 와서 자신의 삶을 반성합니다.,서울특별시 용산구 이촌동 한강대교,37.517492,126.95897,서울,['윤시윤']
7336,drama,You Raise Me Up (유 레이즈 미 업),해밀톤 호텔별관,stay,7회에서 도지혁(박기웅)이 이 술집에서 술을 마신다.,서울특별시 용산구 이태원로27가길 26,37.535099,126.99377,서울,['박기웅']
7337,drama,You Raise Me Up (유 레이즈 미 업),북촌로5가길,playground,8화에서 두라와 데이트를 하며 이 거리를 걷는다.,서울특별시 종로구 북촌로5가길 12-7,37.580327,126.98237,서울,[]


In [13]:
# 촬영지 데이터 : 영화 
media_movie = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_movie_pre.csv", encoding='utf-8-sig')
media_movie

,미디어타입,제목,장소명,장소타입,장소설명,주소,위도,경도,주소_지역명,배우이름
0,movie,1987,임진각,playground,영화 1987 초반부 김윤석이 차례를 지내는 곳,경기도 파주시 문산읍 임진각로 164,37.889542,126.740176,경기,['김윤석']
1,movie,1987,문화공감수정,playground,영화 1987에 나온 부산역 카페,부산광역시 동구 홍곡로 75,35.125713,129.042622,부산,[]
2,movie,1987,연세대학교 신촌캠퍼스,playground,영화 1987에서 김태리가 다닌 대학교,서울특별시 서대문구 연세로 50,37.563756,126.937546,서울,['김태리']
3,movie,1987,민주인권기념관,playground,1987 촬영지이자 박종철이 고문으로 사망한 곳,서울특별시 용산구 한강대로71길 37,37.541025,126.971672,서울,[]
4,movie,1987,천주교 서울대교구 주교좌명동대성당,playground,1987에서 김승훈 신부가 성명서를 발표한 곳,서울특별시 중구 명동길 74,37.563277,126.986827,서울,['김승훈']
...,...,...,...,...,...,...,...,...,...,...
268,movie,택시운전사,경남식당,restaurant,영화 속에서 송강호가 점심 먹은 곳,경상북도 성주군 성주읍 시장길 43-1 103동 102호,35.915624,128.285822,경상,['송강호']
269,movie,택시운전사,칠백장장인국밥,restaurant,"송강호, 고창석이 식사하는 장면이 나온 곳",부산광역시 동래구 미남로 67,35.200047,129.069878,부산,"['송강호', '고창석']"
270,movie,택시운전사,부산영화촬영스튜디오,playground,송강호가 택시 운전하면서 창밖의 배경 CG로 촬영,부산광역시 해운대구 해운대해변로 52,35.162662,129.138461,부산,['송강호']
271,movie,택시운전사,청소면 행정복지센터 앞 거리,playground,이 거리는 80년대 광주민주화운동 배경이 된 곳,충청남도 보령시 청소면 청소큰길 167,36.445337,126.589933,충청,[]


In [14]:
# 촬영지 데이터 : 아티스트 
media_artist = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_artist_pre.csv", encoding='utf-8-sig')
media_artist

,미디어타입,제목,장소명,장소타입,장소설명,주소,위도,경도,주소_지역명,아티스트명
0,artist,2AM,휘닉스 평창 스노우 파크,playground,"우리 결혼했어요 조권, 가인의 데이트 장소",강원도 평창군 봉평면 태기로 174,37.577868,128.323410,강원,"['조권', '가인']"
1,artist,2AM,원마운트,playground,"우리 결혼했어요 진운, 고준희의 데이트 장소",경기도 고양시 일산서구 한류월드로 300,37.664347,126.754379,경기,"['준희', '진운']"
2,artist,2AM,율동공원,playground,"우리 결혼했어요 조권, 가인의 데이트 장소",경기도 성남시 분당구 문정로 72,37.376538,127.152391,경기,"['조권', '가인']"
3,artist,2AM,베르아델승마클럽,playground,"우리 결혼했어요 진운, 고준희의 데이트 코스",경기도 안산시 단원구 부흥로 376,37.212744,126.608469,경기,"['준희', '진운']"
4,artist,2AM,서원아트리움,playground,진운이 아이스버킷챌린지를 한 장소,경기도 파주시 광탄면 서원길 333,37.815938,126.896078,경기,['진운']
...,...,...,...,...,...,...,...,...,...,...
1957,artist,TXT,와이엔,cafe,TO DO X TXT 68화 바리스타편 촬영지,서울특별시 종로구 윤보선길 71 1층,37.579182,126.983312,서울,['TXT']
1958,artist,TXT,롯데백화점 본점,store,TXT의 핸드프린팅이 전시되어 있는 곳,서울특별시 중구 남대문로 81,37.565441,126.980991,서울,['TXT']
1959,artist,TXT,볼링볼링,playground,톡투데이2에서 연준과 태현이 볼링을 쳤던 장소,서울특별시 중구 청계천로 400 롯데캐슬베네치아 메가몰동 지하1층,37.570766,127.019836,서울,[]
1960,artist,TXT,라마다 제주함덕호텔,stay,220605 위버스에 태현이 업로드한 호텔,제주특별자치도 제주시 조천읍 신북로 470,33.543255,126.659864,제주,['태현']


In [15]:
# 숙박 데이터 
hotel = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/숙박 데이터/hotel_pre.csv", encoding='utf-8-sig')
hotel

,Unnamed: 0,LDGS_NM,LDGS_ADDR,LDGS_ROAD_NM_ADDR,GSRM_SCALE_CN,LDGS_GRAD_VALUE,LDGMNT_TY_NM,LDGS_AVRG_PRC,LDGS_MIN_PRC,LDGS_MXMM_PRC,LDGS_AVRG_SCORE_CO,위도,경도
0,0,stx 리조트,경상,경북 문경시 농암면 청화로 509,200개 이상,성급없음,휴양콘도미니엄업,154079,105000,270600,9,36.569324,127.964360
1,1,경주 코오롱호텔,경상,경북 경주시 불국로 289-17,200개 이상,4성,관광호텔업,231518,74000,924000,9,35.787693,129.321973
2,2,곤지암리조트,경기,경기 광주시 도척면 도척윗로 278,200개 이상,성급없음,숙박업(생활),276738,101823,501400,9,37.337559,127.296189
3,3,그랜드 워커힐 서울,서울,서울특별시 광진구 워커힐로 177,200개 이상,5성,관광호텔업,418933,157700,762300,8,37.558629,127.111707
4,4,그랜드 조선 부산,부산,부산 해운대구 해운대해변로 292,200개 이상,5성,관광호텔업,615915,219999,990000,9,35.160066,129.163128
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,125,호텔현대 바이 라한 목포,전라,전남 영암군 삼호읍 대불로 91,200개 이상,4성,관광호텔업,208659,99000,604056,9,34.733475,126.387321
126,126,휘닉스 섭지코지,제주,제주특별자치도 서귀포시 성산읍 섭지코지로 107,200개 이상,성급없음,숙박업(생활),451920,181751,990909,9,33.426042,126.926443
127,127,휘닉스평창 호텔,강원,강원 평창군 봉평면 태기로 174,200개 이하,4성,관광호텔업,338300,72000,790000,10,37.582542,128.326741
128,128,힐튼 호텔 경주,경상,경북 경주시 보문로 484-7,200개 이상,5성,관광호텔업,458950,160000,825220,8,35.839913,129.285948


In [16]:
hotel.rename(columns={
    'Unnamed: 0': '번호',
    'LDGS_NM': '숙박명',
    'LDGS_ADDR': '지역명',
    'LDGS_ROAD_NM_ADDR': '도로명주소',
    'GSRM_SCALE_CN': '규모',
    'LDGS_GRAD_VALUE': '성급',
    'LDGMNT_TY_NM': '숙박유형',
    'LDGS_AVRG_PRC': '평균가격',
    'LDGS_MIN_PRC': '최소가격',
    'LDGS_MXMM_PRC': '최대가격',
    'LDGS_AVRG_SCORE_CO': '평균평점',
    '위도': '위도',
    '경도': '경도'
}, inplace=True)
hotel.columns

Index(['번호', '숙박명', '지역명', '도로명주소', '규모', '성급', '숙박유형', '평균가격', '최소가격', '최대가격',
       '평균평점', '위도', '경도'],
      dtype='object')

* 추천 알고리즘 구현 
    * 카카오맵 지도의 웹사이트 링크는 저장된 html을 열고, 클릭해야 해당 링크로 들어가짐

#### ** 알고리즘 고도화**

* 거리 기반 추천 시스템으로 코드 수정 


    * geopy.distance.geodesic() 사용해서 거리 계산

    * 촬영지 좌표 평균값을 중심점으로 사용

    * 각 장소(restaurant, cafe, 숙박)에 대해 거리 계산 후 정렬하여 .head(3)



In [18]:
import pandas as pd
import folium
from geopy.distance import geodesic
from IPython.display import display

# ---------------------- [1. 데이터 불러오기 및 전처리] ----------------------
media_drama = pd.read_csv("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_drama_pre.csv", encoding='utf-8-sig')
media_movie = pd.read_csv("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_movie_pre.csv", encoding='utf-8-sig')
media_artist = pd.read_csv("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_artist_pre.csv", encoding='utf-8-sig')
kakao_food = pd.read_csv("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/카카오맵 맛집 크롤링_전처리_데이터/kakao_food_geocode.csv", encoding="utf-8-sig")
hotel_df = pd.read_csv("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/숙박 데이터/hotel_pre.csv", encoding='utf-8-sig')

hotel_df.columns = hotel_df.columns.str.strip()
hotel_df.rename(columns={
    'Unnamed: 0': '번호',
    'LDGS_NM': '숙박명',
    'LDGS_ADDR': '지역명',
    'LDGS_ROAD_NM_ADDR': '도로명주소',
    'GSRM_SCALE_CN': '규모',
    'LDGS_GRAD_VALUE': '성급',
    'LDGMNT_TY_NM': '숙박유형',
    'LDGS_AVRG_PRC': '평균가격',
    'LDGS_MIN_PRC': '최소가격',
    'LDGS_MXMM_PRC': '최대가격',
    'LDGS_AVRG_SCORE_CO': '평균평점',
}, inplace=True)

# ---------------------- [2. 사용자 입력] ----------------------
media_type = input("검색할 분야 (drama, movie, artist): ").strip()
name = input("배우 또는 아티스트 이름 입력: ").strip()
region = input("지역 선택 (예: 경기): ").strip()

# ---------------------- [3. 데이터 선택 및 필터링] ----------------------
if media_type == 'drama':
    df = media_drama
elif media_type == 'movie':
    df = media_movie
elif media_type == 'artist':
    df = media_artist
else:
    raise ValueError("media_type은 drama, movie, artist 중 하나여야 합니다.")

# 이름, 지역 필터링
if media_type in ['drama', 'movie']:
    filtered = df[df['배우이름'].str.contains(name, na=False) & (df['주소_지역명'] == region)]
else:
    filtered = df[df['아티스트명'].str.contains(name, na=False) & (df['주소_지역명'] == region)]

# ❗ 빈 경우 처리
if filtered.empty:
    print("해당 배우/아티스트와 지역 조합으로 데이터가 없습니다.")
    m = folium.Map(location=[37.5665, 126.9780], zoom_start=10)  # 서울
    display(m)
    exit()

# ---------------------- [4. 촬영지 정제 및 중심좌표 계산] ----------------------
recommend_place = filtered.groupby('장소타입').apply(lambda x: x.head(3)).reset_index(drop=True)

# ---------------------- [★ 선택형 추천 기능 추가 위치] ----------------------
# unique 장소 목록 출력용
display_df = recommend_place[['제목', '장소명', '장소타입']].drop_duplicates().reset_index(drop=True)

# 출력
print("🎬 추천 촬영지 목록 (1개만 선택):")
for i, row in display_df.iterrows():
    print(f"{i+1}. 제목: {row['제목']}, 장소: {row['장소명']}, 타입: {row['장소타입']}")

# 선택
selected_index = input(f"\n추천받고 싶은 촬영지 번호를 입력하세요 (1 ~ {len(display_df)}): ").strip()

# 정수 변환 및 유효성 검사
if selected_index.isdigit():
    selected_index = int(selected_index) - 1
    if 0 <= selected_index < len(display_df):
        selected_row = display_df.loc[selected_index]
        # 선택된 장소 기준으로 recommend_place 필터링
        recommend_place = recommend_place[
            (recommend_place['제목'] == selected_row['제목']) &
            (recommend_place['장소명'] == selected_row['장소명']) &
            (recommend_place['장소타입'] == selected_row['장소타입'])
        ].reset_index(drop=True)
    else:
        print("❗입력한 번호가 유효하지 않습니다.")
        exit()
else:
    print("❗숫자로 입력해주세요.")
    exit()

results = []
for idx, row in recommend_place.iterrows():
    place_name = row['장소명']
    place_lat = row['위도']
    place_lng = row['경도']
    place_type = row['장소타입']
    place_title = row.get('제목', '제목 없음')

    def calc_distance_to_place(r):
        try:
            return geodesic((place_lat, place_lng), (r['위도'], r['경도'])).meters
        except:
            return float('inf')

    kakao_food['거리'] = kakao_food.apply(calc_distance_to_place, axis=1)
    hotel_df['거리'] = hotel_df.apply(calc_distance_to_place, axis=1)

    closest_rests = kakao_food[(kakao_food['카테고리'] == 'restaurant') & (kakao_food['지역명'] == region)].sort_values('거리').head(3)
    closest_cafes = kakao_food[(kakao_food['카테고리'] == 'cafe') & (kakao_food['지역명'] == region)].sort_values('거리').head(3)
    closest_hotels = hotel_df[hotel_df['지역명'] == region].sort_values('거리').head(3)

    results.append({
        '촬영지 제목': place_title,
        '장소명': place_name,
        '장소타입': place_type,
        '맛집 목록': closest_rests['식당이름'].tolist() if not closest_rests.empty else ['-'],
        '맛집 거리(m)': [round(d, 2) for d in closest_rests['거리'].tolist()] if not closest_rests.empty else ['-'],
        '카페 목록': closest_cafes['식당이름'].tolist() if not closest_cafes.empty else ['-'],
        '카페 거리(m)': [round(d, 2) for d in closest_cafes['거리'].tolist()] if not closest_cafes.empty else ['-'],
        '숙소 목록': closest_hotels['숙박명'].tolist() if not closest_hotels.empty else ['-'],
        '숙소 거리(m)': [round(d, 2) for d in closest_hotels['거리'].tolist()] if not closest_hotels.empty else ['-'],
    })

recommend_detail = pd.DataFrame(results)
print("\n📍 각 촬영지별 주변 추천 (3개씩):")
display(recommend_detail)

<ipython-input-18-c45a8a736d89>:57: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  recommend_place = filtered.groupby('장소타입').apply(lambda x: x.head(3)).reset_index(drop=True)


🎬 추천 촬영지 목록 (1개만 선택):
1. 제목: 방탄소년단, 장소: 학동공원, 타입: playground
2. 제목: 방탄소년단, 장소: 국립중앙박물관, 타입: playground
3. 제목: 방탄소년단, 장소: 해쉬더우드, 타입: playground
4. 제목: 방탄소년단, 장소: 유정식당, 타입: restaurant
5. 제목: 방탄소년단, 장소: 본앤브레드, 타입: restaurant
6. 제목: 방탄소년단, 장소: 열봉부엌, 타입: restaurant

📍 각 촬영지별 주변 추천 (3개씩):


,촬영지 제목,장소명,장소타입,맛집 목록,맛집 거리(m),카페 목록,카페 거리(m),숙소 목록,숙소 거리(m)
0,방탄소년단,국립중앙박물관,playground,"[롸카두들 이태원점, 돌판장, 아노브 이태원본점]","[1390.59, 1396.42, 1522.3]","[맥심플랜트, 프리츠 도화점, 마일스톤커피]","[2329.46, 3358.26, 3894.57]","[노보텔 앰배서더 서울 용산, 그랜드 하얏트 서울, 서울신라호텔]","[1948.43, 2248.41, 4452.73]"


In [19]:
import pandas as pd

# 1) 맛집만 풀기
rows_rest = []
for _, row in recommend_detail.iterrows():
    places = row['맛집 목록']
    distances = row['맛집 거리(m)']
    for place, dist in zip(places, distances):
        rows_rest.append({
            '촬영지 제목': row['촬영지 제목'],
            '촬영지 장소명': row['장소명'],
            '맛집명': place,
            '거리(m)': dist
        })
extra_restaurant = pd.DataFrame(rows_rest)

# 2) 카페만 풀기
rows_cafe = []
for _, row in recommend_detail.iterrows():
    places = row['카페 목록']
    distances = row['카페 거리(m)']
    for place, dist in zip(places, distances):
        rows_cafe.append({
            '촬영지 제목': row['촬영지 제목'],
            '촬영지 장소명': row['장소명'],
            '카페명': place,
            '거리(m)': dist
        })
extra_cafe = pd.DataFrame(rows_cafe)

# 3) 숙소만 풀기
rows_hotel = []
for _, row in recommend_detail.iterrows():
    places = row['숙소 목록']
    distances = row['숙소 거리(m)']
    for place, dist in zip(places, distances):
        rows_hotel.append({
            '촬영지 제목': row['촬영지 제목'],
            '촬영지 장소명': row['장소명'],
            '숙소명': place,
            '거리(m)': dist
        })
recommend_hotel = pd.DataFrame(rows_hotel)

# 예: kakao_food와 extra_restaurant 조인
extra_restaurant = extra_restaurant.merge(
    kakao_food[['식당이름', '카테고리', '별점', '리뷰 수', '웹사이트', '주소', '위도', '경도']],
    left_on='맛집명', right_on='식당이름', how='left'
)

extra_cafe = extra_cafe.merge(
    kakao_food[['식당이름', '위도', '경도', '카테고리', '별점', '리뷰 수', '주소', '웹사이트']],
    left_on='카페명', right_on='식당이름', how='left'
)

recommend_hotel = recommend_hotel.merge(
    hotel_df[['숙박명', '도로명주소', '성급', '최소가격', '최대가격', '평균평점', '위도', '경도']],
    left_on='숙소명', right_on='숙박명', how='left'
)


In [20]:
center_point = (recommend_place['위도'].mean(), recommend_place['경도'].mean())

# 지도 생성
m = folium.Map(location=center_point, zoom_start=12)

# 🎬 촬영지 마커
for _, row in recommend_place.iterrows():
    popup = folium.Popup(f"""
        <b>{row.get('제목', '제목 없음')}</b><br>
        장소명: {row.get('장소명', '-') }<br>
        장소타입: {row.get('장소타입', '-') }<br>
        설명: {row.get('장소설명', '-') }<br>
        주소: {row.get('주소', '-') }
    """, max_width=300)
    folium.Marker(
        [row['위도'], row['경도']], 
        popup=popup,
        icon=folium.Icon(icon='video-camera', prefix='fa', color='blue')
    ).add_to(m)

# 🍽 맛집 마커 (red, cutlery)
for _, row in extra_restaurant.iterrows():
    리뷰수 = int(row.get('리뷰 수', 0))
    website = row.get('웹사이트', '')
    if not pd.isna(website) and not website.startswith("http"):
        website = "https://" + website

    html = f"""
        <b>{row.get('식당이름', '식당명 없음')}</b><br>
        카테고리: {row.get('카테고리', '-') }<br>
        별점: {row.get('별점', '-') }<br>
        리뷰수: {리뷰수}<br>
        주소: {row.get('주소', '-') }<br>
        <a href="{website}" target="_blank">웹사이트</a>
    """
    popup = folium.Popup(folium.Html(html, script=True), max_width=300)

    folium.Marker(
        [row['위도'], row['경도']],
        popup=popup,
        icon=folium.Icon(icon='cutlery', prefix='fa', color='red')
    ).add_to(m)

# ☕ 카페 마커 (green, coffee)
for _, row in extra_cafe.iterrows():
    리뷰수 = int(row.get('리뷰 수', 0))
    website = row.get('웹사이트', '')
    if not pd.isna(website) and not website.startswith("http"):
        website = "https://" + website

    html = f"""
        <b>{row.get('식당이름', '카페명 없음')}</b><br>
        카테고리: {row.get('카테고리', '-') }<br>
        별점: {row.get('별점', '-') }<br>
        리뷰수: {리뷰수}<br>
        주소: {row.get('주소', '-') }<br>
        <a href="{website}" target="_blank">웹사이트</a>
    """
    popup = folium.Popup(folium.Html(html, script=True), max_width=300)

    folium.Marker(
        [row['위도'], row['경도']],
        popup=popup,
        icon=folium.Icon(icon='coffee', prefix='fa', color='orange')
    ).add_to(m)


# 🏨 숙박 마커
for _, row in recommend_hotel.iterrows():
    popup = folium.Popup(f"""
        <b>{row.get('숙박명', '이름 없음')}</b><br>
        도로명 주소: {row.get('도로명주소', '-') }<br>
        성급: {row.get('성급', '-') }<br>
        최소가격: {row.get('최소가격', '-') }원<br>
        최대가격: {row.get('최대가격', '-') }원<br>
        평균평점: {row.get('평균평점', '-') }
    """, max_width=300)
    folium.Marker(
        [row['위도'], row['경도']],
        popup=popup,
        icon=folium.Icon(icon='bed', prefix='fa', color='green')
    ).add_to(m)

# 지도 출력 및 저장
display(m)

save_path = "C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/지도/recommendation_map_re.html"
m.save(save_path)
print(f"지도가 저장되었습니다: {save_path}")


지도가 저장되었습니다: C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/지도/recommendation_map_re.html
